# Random Forest

### In this NB I shall be following along with  Jeremy H as he goes through the intro to machine learning lessons. We start with the Random Forest algorithm.

The lesson we're following today is from the FastAi ML MOOC which ran in late 2018, it is taught by Jeremy H and Rachel, it is ideal for anyone who wants to break in to Machine learning or someone wanting to learn about ML Algorithms.

Dataset
The Dataset is from Kaggle, its from an old competition about the Bulldozzers sales. We have four years worth of data and the aim is to predicate the __Sale Price__ for a future time.

#### Imports

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics
import feather

##### Importing the data 

In [3]:
PATH = "excel/Train.csv"

###### A few things to Note:
* FastAi is a library coded by the FastAi team.
* The library is built on top of the famous open source libraries like SK_Learn and Pandas.
* Its advisable to check out the FastAi forum whenever you find yourself stuck.


### About the Algorithm

Random Forest is one of the most important algorithms in data science, it is pretty much the go to technic for the all problems. Understanding this algorithm will put you in a very good position for rest of your time as a data scientist.

Random Forest is based of building a tree from the data, you build this tree using features. we then build a forest of these tree and use the averages to pick out insights. 

In [4]:
%%time
# This code takes a while depending on your 
# We can set it to low memory to alert pandas of the capability of our machine
# We also Pasrse the target feature as dates

raw_data = pd.read_csv("excel/Train.csv", low_memory=False, 
                     parse_dates=["saledate"])

Wall time: 1min 34s


We check out the head of the data to see how it looks like, doing this can be insightful especially at the beginning of the analysis

In [5]:
raw_data.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,2011-05-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,2009-07-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


What we see from the above table is that we have fifty three features and the data has both numerical and categorical features.
Having a look at the head is go to method to call for a quick look but for insight we need to look at our data in various ways before we play with, this helps us rule out what kind of techniques will work and how to go about playing with our data.

For this we need a function that will us with this task, the **_display-all()_** functions helps us to see the data in different way, we use pandas to do this.

In [6]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [7]:
raw_data.tail()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
401120,6333336,10500,1840702,21439,149,1.0,2005,NaN,NaN,2011-11-02,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401121,6333337,11000,1830472,21439,149,1.0,2005,NaN,NaN,2011-11-02,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401122,6333338,11500,1887659,21439,149,1.0,2005,NaN,NaN,2011-11-02,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401123,6333341,9000,1903570,21435,149,2.0,2005,NaN,NaN,2011-10-25,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401124,6333342,7750,1926965,21435,149,2.0,2005,NaN,NaN,2011-10-25,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN


In [8]:
display_all(raw_data.tail())

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Drive_System,Enclosure,Forks,Pad_Type,Ride_Control,Stick,Transmission,Turbocharged,Blade_Extension,Blade_Width,Enclosure_Type,Engine_Horsepower,Hydraulics,Pushblock,Ripper,Scarifier,Tip_Control,Tire_Size,Coupler,Coupler_System,Grouser_Tracks,Hydraulics_Flow,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
401120,6333336,10500,1840702,21439,149,1.0,2005,NaN,NaN,2011-11-02,35NX2,35,NX,2,NaN,Mini,"Hydraulic Excavator, Track - 3.0 to 4.0 Metric...",Maryland,TEX,Track Excavators,NaN,EROPS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Auxiliary,NaN,NaN,NaN,NaN,NaN,None or Unspecified,NaN,NaN,NaN,Steel,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401121,6333337,11000,1830472,21439,149,1.0,2005,NaN,NaN,2011-11-02,35NX2,35,NX,2,NaN,Mini,"Hydraulic Excavator, Track - 3.0 to 4.0 Metric...",Maryland,TEX,Track Excavators,NaN,EROPS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,NaN,NaN,NaN,NaN,NaN,None or Unspecified,NaN,NaN,NaN,Steel,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401122,6333338,11500,1887659,21439,149,1.0,2005,NaN,NaN,2011-11-02,35NX2,35,NX,2,NaN,Mini,"Hydraulic Excavator, Track - 3.0 to 4.0 Metric...",Maryland,TEX,Track Excavators,NaN,EROPS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Auxiliary,NaN,NaN,NaN,NaN,NaN,None or Unspecified,NaN,NaN,NaN,Steel,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401123,6333341,9000,1903570,21435,149,2.0,2005,NaN,NaN,2011-10-25,30NX,30,NX,NaN,NaN,Mini,"Hydraulic Excavator, Track - 2.0 to 3.0 Metric...",Florida,TEX,Track Excavators,NaN,EROPS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,NaN,NaN,NaN,NaN,NaN,None or Unspecified,NaN,NaN,NaN,Steel,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
401124,6333342,7750,1926965,21435,149,2.0,2005,NaN,NaN,2011-10-25,30NX,30,NX,NaN,NaN,Mini,"Hydraulic Excavator, Track - 2.0 to 3.0 Metric...",Florida,TEX,Track Excavators,NaN,EROPS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,NaN,NaN,NaN,NaN,NaN,None or Unspecified,NaN,NaN,NaN,Steel,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN


In [9]:
display_all(raw_data.tail().T)

,401120,401121,401122,401123,401124
SalesID,6333336,6333337,6333338,6333341,6333342
SalePrice,10500,11000,11500,9000,7750
MachineID,1840702,1830472,1887659,1903570,1926965
ModelID,21439,21439,21439,21435,21435
datasource,149,149,149,149,149
auctioneerID,1,1,1,2,2
YearMade,2005,2005,2005,2005,2005
MachineHoursCurrentMeter,NaN,NaN,NaN,NaN,NaN
UsageBand,NaN,NaN,NaN,NaN,NaN
saledate,2011-11-02 00:00:00,2011-11-02 00:00:00,2011-11-02 00:00:00,2011-10-25 00:00:00,2011-10-25 00:00:00


In [10]:
display_all(raw_data.describe(include='all').T)

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
SalesID,401125,NaN,NaN,NaN,NaN,NaN,1.91971e+06,909021,1.13925e+06,1.41837e+06,1.63942e+06,2.24271e+06,6.33334e+06
SalePrice,401125,NaN,NaN,NaN,NaN,NaN,31099.7,23036.9,4750,14500,24000,40000,142000
MachineID,401125,NaN,NaN,NaN,NaN,NaN,1.2179e+06,440992,0,1.0887e+06,1.27949e+06,1.46807e+06,2.48633e+06
ModelID,401125,NaN,NaN,NaN,NaN,NaN,6889.7,6221.78,28,3259,4604,8724,37198
datasource,401125,NaN,NaN,NaN,NaN,NaN,134.666,8.96224,121,132,132,136,172
auctioneerID,380989,NaN,NaN,NaN,NaN,NaN,6.55604,16.9768,0,1,2,4,99
YearMade,401125,NaN,NaN,NaN,NaN,NaN,1899.16,291.797,1000,1985,1995,2000,2013
MachineHoursCurrentMeter,142765,NaN,NaN,NaN,NaN,NaN,3457.96,27590.3,0,0,0,3025,2.4833e+06
UsageBand,69639,3,Medium,33985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
saledate,401125,3919,2009-02-16 00:00:00,1932,1989-01-17 00:00:00,2011-12-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
raw_data.SalePrice = np.log(raw_data.SalePrice)

In [12]:
raw_data.SalePrice

0         11.097410
1         10.950807
2          9.210340
3         10.558414
4          9.305651
5         10.184900
6          9.952278
7         10.203592
8          9.975808
9         11.082143
10        10.085809
11        10.021271
12        10.491274
13        10.325482
14        10.239960
15         9.852194
16         9.510445
17         9.159047
18         9.433484
19         9.350102
20        10.621327
21        10.448715
22        10.165852
23        11.198215
24        10.404263
25         9.433484
26         9.648595
27        10.878047
28        10.736397
29        11.396392
            ...    
401095     9.259131
401096     9.210340
401097     9.047821
401098     9.259131
401099     9.305651
401100     9.259131
401101     9.210340
401102     9.259131
401103     9.433484
401104     9.259131
401105     9.210340
401106     9.259131
401107     9.433484
401108     9.259131
401109     9.259131
401110     9.259131
401111     9.210340
401112     9.259131
401113     9.259131


## Initial Processing 

The data is its current condition we can't really do much with it even for exploratory analysis, we need to process it first before we can do something.

**Note:** If you're not familiar with a function you can just type two question marks and the function to see what the function does, I use it to find out more about a function in the next cell.

The function in question is from FastAi library we use it to add convert a column of df from a datetime64 to many columns containing the information from the date.

In [13]:
??add_datepart

In [14]:
raw_data.saledate.head(3)

0   2006-11-16
1   2004-03-26
2   2004-02-26
Name: saledate, dtype: datetime64[ns]

After applying the function

In [15]:
add_datepart(raw_data,'saledate')
raw_data.saleYear.head()

0    2006
1    2004
2    2004
3    2011
4    2009
Name: saleYear, dtype: int64

Splitting the data this way will help us to get a better insight in the __saledates__ feature, if you use the Jupyter magic on the function you would have seen that the function splits the feature helps us to extract a lot more data that we would not have had, had we just use it in its pure form. We also have new columns as a result of calling this functions on the data. 

A quick look at the way our dataset looks after the **add_datepart** functions.
Notice we have added more columns to our table.

In [16]:
raw_data.head(3)

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,saleDay,saleDayofweek,saleDayofyear,saleIs_month_end,saleIs_month_start,saleIs_quarter_end,saleIs_quarter_start,saleIs_year_end,saleIs_year_start,saleElapsed
0,1139246,11.097410,999089,3157,121,3.0,2004,68.0,Low,521D,...,16,3,320,False,False,False,False,False,False,1163635200
1,1139248,10.950807,117657,77,121,3.0,1996,4640.0,Low,950FII,...,26,4,86,False,False,False,False,False,False,1080259200
2,1139249,9.210340,434808,7009,121,3.0,2001,2838.0,High,226,...,26,3,57,False,False,False,False,False,False,1077753600


The next this to deal with the state of our dataset, for example we would like our dataset to have all the features in a numerical form, however we have some columns which are strings, so we need to deal with these columns before we start training. 

We shall use the train cats function also from the FirstAi library what it does, it changes any columns of strings in a panda's dataframe to a column of categorical values. This applies the changes inplace.

In [17]:
train_cats(raw_data)

In [18]:
raw_data.UsageBand.cat.categories

Index(['High', 'Low', 'Medium'], dtype='object')

Lets take another quick look at our dataset to see what we have done so far 

In [19]:
raw_data.head(5)

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,saleDay,saleDayofweek,saleDayofyear,saleIs_month_end,saleIs_month_start,saleIs_quarter_end,saleIs_quarter_start,saleIs_year_end,saleIs_year_start,saleElapsed
0,1139246,11.097410,999089,3157,121,3.0,2004,68.0,Low,521D,...,16,3,320,False,False,False,False,False,False,1163635200
1,1139248,10.950807,117657,77,121,3.0,1996,4640.0,Low,950FII,...,26,4,86,False,False,False,False,False,False,1080259200
2,1139249,9.210340,434808,7009,121,3.0,2001,2838.0,High,226,...,26,3,57,False,False,False,False,False,False,1077753600
3,1139251,10.558414,1026470,332,121,3.0,2001,3486.0,High,PC120-6E,...,19,3,139,False,False,False,False,False,False,1305763200
4,1139253,9.305651,1057373,17311,121,3.0,2007,722.0,Medium,S175,...,23,3,204,False,False,False,False,False,False,1248307200


Two things to note here; we still have strings in our dataset despite calling the train function which was supposed to change them into numerical values, we also have boolean values in our dataset, these will be represented as 0 and 1 during the processing even though at the moment Pandas shows it to us as Strings, they are really boolean values ie 0 and 1 version will be used during the processing.

In [20]:
raw_data.UsageBand = raw_data.UsageBand.cat.codes

We are still not quite done, for instance we have lots of missing values, which we can't pass directly to a random forest. There are various ways to deal with null values in a dataset, here we shall employ the fillna values method which fills null values with approximate values, this method can backfire so care must be taken when choosing a method to deal with null values.

In [21]:
display_all(raw_data.isnull().sum().sort_index()/len(raw_data))

Backhoe_Mounting            0.803872
Blade_Extension             0.937129
Blade_Type                  0.800977
Blade_Width                 0.937129
Coupler                     0.466620
Coupler_System              0.891660
Differential_Type           0.826959
Drive_System                0.739829
Enclosure                   0.000810
Enclosure_Type              0.937129
Engine_Horsepower           0.937129
Forks                       0.521154
Grouser_Tracks              0.891899
Grouser_Type                0.752813
Hydraulics                  0.200823
Hydraulics_Flow             0.891899
MachineHoursCurrentMeter    0.644089
MachineID                   0.000000
ModelID                     0.000000
Pad_Type                    0.802720
Pattern_Changer             0.752651
ProductGroup                0.000000
ProductGroupDesc            0.000000
ProductSize                 0.525460
Pushblock                   0.937129
Ride_Control                0.629527
Ripper                      0.740388
S

Doing this kind of processing is bit risky on the actual dataset especially when you're working on actuall data, it is recommeded to make a copy of the data and play with this copy so permanet changes wont affect the original data especially during the processing real damage can be done.

In [22]:
#To make a temp 
os.makedirs('tmp',exist_ok=True)
raw_data.to_feather('tmp/bulldozers-raw1')

From now on we shall be using this copy, there are two main advantages to use this copy, its quicker incase our notebook goes down we can just begin were we stop. Its also safer in terms of things going bad.

### Pre-processing

In [23]:
raw_data = feather.read_dataframe('tmp/bulldozers-raw1')

Now we can deal with the other categorical features without worrying that they will disrupt our original dataset, we still havent dealt with the null values in the dataset.
We're going to use the **proc_df()** function again from FastAi to replace the categorical data with numerical values in order to handle missing values, we shall also split the dependant variable into a seperable variable.

In [24]:
df, y, nas = proc_df(raw_data, 'SalePrice')

A quick lookat the state of our columns after calling the proc_df functions 

In [25]:
df.columns

Index(['SalesID', 'MachineID', 'ModelID', 'datasource', 'auctioneerID',
       'YearMade', 'MachineHoursCurrentMeter', 'UsageBand', 'fiModelDesc',
       'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries', 'fiModelDescriptor',
       'ProductSize', 'fiProductClassDesc', 'state', 'ProductGroup',
       'ProductGroupDesc', 'Drive_System', 'Enclosure', 'Forks', 'Pad_Type',
       'Ride_Control', 'Stick', 'Transmission', 'Turbocharged',
       'Blade_Extension', 'Blade_Width', 'Enclosure_Type', 'Engine_Horsepower',
       'Hydraulics', 'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control',
       'Tire_Size', 'Coupler', 'Coupler_System', 'Grouser_Tracks',
       'Hydraulics_Flow', 'Track_Type', 'Undercarriage_Pad_Width',
       'Stick_Length', 'Thumb', 'Pattern_Changer', 'Grouser_Type',
       'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
       'Differential_Type', 'Steering_Controls', 'saleYear', 'saleMonth',
       'saleWeek', 'saleDay', 'saleDayofweek', 'saleDayofyear',
       'saleI

In [26]:
# a quick look at the head again
df.tail(4)

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,fiBaseModel,...,saleDayofyear,saleIs_month_end,saleIs_month_start,saleIs_quarter_end,saleIs_quarter_start,saleIs_year_end,saleIs_year_start,saleElapsed,auctioneerID_na,MachineHoursCurrentMeter_na
401121,6333337,1830472,21439,149,1.0,2005,0.0,-1,657,207,...,306,False,False,False,False,False,False,1320192000,False,True
401122,6333338,1887659,21439,149,1.0,2005,0.0,-1,657,207,...,306,False,False,False,False,False,False,1320192000,False,True
401123,6333341,1903570,21435,149,2.0,2005,0.0,-1,483,159,...,298,False,False,False,False,False,False,1319500800,False,True
401124,6333342,1926965,21435,149,2.0,2005,0.0,-1,483,159,...,298,False,False,False,False,False,False,1319500800,False,True


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 66 columns):
SalesID                        401125 non-null int64
MachineID                      401125 non-null int64
ModelID                        401125 non-null int64
datasource                     401125 non-null int64
auctioneerID                   401125 non-null float64
YearMade                       401125 non-null int64
MachineHoursCurrentMeter       401125 non-null float64
UsageBand                      401125 non-null int8
fiModelDesc                    401125 non-null int16
fiBaseModel                    401125 non-null int16
fiSecondaryDesc                401125 non-null int16
fiModelSeries                  401125 non-null int8
fiModelDescriptor              401125 non-null int16
ProductSize                    401125 non-null int8
fiProductClassDesc             401125 non-null int8
state                          401125 non-null int8
ProductGroup                   401125 non-

We can confrim that we no longer have null and data of the type String in our dataset.

We now have our d,ata in a form that can be used to pass it in a random forest, we shall be using Sklearn model **RandomForestRegressor**

## Base Model

##### Unrealistic model

In [28]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(df, y)
m.score(df,y)

C:\Users\Shaki\Anaconda3\envs\fastai-cpu\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Wall time: 58 s


0.9830515284596412

What have we done here?
* 1. We called our model the default is ten trees in Sklearn
* 2. We fit our model to the data 
* 3. We called th score function to see how we did.

Bear in mind that we havent done any thing other than processing the data, the .98 value above represents the RSquared, this is the same value we use to judge how well the model did.
But this sis not enogh infact this is not how you would do it in a real scenario, because we used all our data dring the fitting process we have no way of knowing how our model will perform on unseen data.

This is where the idea of seperating data comes in, the idea is we seperate data into two sets the training set and the validaton set, we do this because not doing so might risk us to over fit as we did in the earlier model. The only way we can know if we're overfiting or not is to try our model on new data to see if we get the same results as we did before.

**Spliting the data:** 
We're going to define a split function, the function will split our data into two copies, the first copy starts from the beginnig to some random n and the other copy begins from n to the end of dataset.
We set out validation set to be 12000, its 12000 because this is the same value that was set byb kaggle in the original competition. We also set our training set as the rest of the data after subtracting the validation set.

In [29]:
def split_vals(a,n): 
    
    return a[:n].copy(), a[n:].copy()

n_valid = 12000  # same as Kaggle's test set size
n_trn = len(df)-n_valid
raw_train, raw_valid = split_vals(raw_data, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape

((389125, 66), (389125,), (12000, 66))

After spliting we can try to build another model but this time we have split the data, we're in a better position to fight against biases and overfiting problems.  

In [30]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [31]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

C:\Users\Shaki\Anaconda3\envs\fastai-cpu\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Wall time: 57.1 s
[0.09047465659376061, 0.2463240209038053, 0.9828924147501904, 0.891641869283041]


#### Speeding things up

One of the main advertages of working in a jupyter notebook is that it allows us to interact with our code on the go, this is normally easy when were working with a small dataset however it becomes increasly difficult on lager datasets, data sciencetist are therfore always on the look out for how to speed things up.
One of the ways that the have comeup with is selecting a subset of data from the original data and building a model on a subset then try the findings on the whole dataset later. 

In [32]:
df_trn, y_trn, nas = proc_df(raw_data, 'SalePrice', subset=30000, na_dict=nas)
X_train, _ = split_vals(df_trn, 20000)
y_train, _ = split_vals(y_trn, 20000)

In [33]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

C:\Users\Shaki\Anaconda3\envs\fastai-cpu\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Wall time: 2.12 s
[0.11173537338161475, 0.3695399138220018, 0.9731573941148652, 0.7561230918838855]


As you can see in the above results, even though we worked on a smaller dataset we still managed to get close to the same result as we did earlier when we still had the whole dataset. The main thing I would like to draw your attention to is how fast it ran compared to the first model.

|DataSet | Model | **Speed(time_secs)**|
|--------|-------|---------------------|
|Whole   |RFR    |40 secs              |
|Subset  | RFR   | 1.4 secs            |

## Single tree

In order to fully understand Random Forest Regression, we have to have an idea of where it gets its name and how it come about. We start with Forest a word that most of us will be familiar with, the use of this word gives us a sense that the a model has something to do with trees, which it does, the model its self id a derived from another popular model called __Decision Tree__, this model as the name suggest builds a tree using data features selecting the best metric for the feature in each branch until it reaches to the last leaf.

As we mentioned earlier, we use scikit learn library, in scikit learn the trees are referred to as estimators, the depth is how deep do you want the tree to go and the number of jobs is how many times you want the model to run.

In [34]:
m = RandomForestRegressor(n_estimators=1, max_depth=3, bootstrap=False, n_jobs=-1) 
%time m.fit(X_train, y_train)
print_score(m)

Wall time: 350 ms
[0.5204350314886235, 0.5812131158431854, 0.4176603650362011, 0.3967201965027739]


In [35]:
m = RandomForestRegressor(n_estimators=1, bootstrap=False, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

[8.702335715267317e-17, 0.5609094257638505, 1.0, 0.4381330973642525]


### Bagging Method

#### Introduction to Bagging method

Bagging method aslo known as is the idea of having multiple trees to get more generalized results. As way of  example we can imagine having 